# Import Library

In [0]:
import sklearn
sklearn.__version__
import scipy

import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from graphviz import Digraph

# Conditional Causality


## Input Data and Graph name change

In [0]:
df = pd.read_csv("combined-nonlinear.csv", header='infer')
# g = Digraph('G', filename='boosting_causality.gv', strict=True)
n = df.shape[0]
k = df.shape[1]

## Functions: regression, boosting, causality_test

In [0]:
def regression(df, x_name, y_name, maxlag):
    data = df
    df_list = []     
    
    v = 0.01
    # add lagged columns of current x variable as x_name
    for lag in range(1,maxlag+1):
        data['{}_{}'.format(x_name,str(lag))] = data['{}'.format(x_name)].shift(lag)
        df_list.append(data['{}_{}'.format(x_name,str(lag))])

    # create test dataframe X, and y
    X = pd.concat(df_list,axis = 1)  
    y = data[y_name]

    # remove NaN rows, the number of removal is maxlag
    X = X.iloc[maxlag:]
    y = y.iloc[maxlag:]
#     print(y)
    print("y is ")
    print(y_name)
    print("X are")
    print(x_name)
    # build regression reg_y, X→y
    reg_y = DecisionTreeRegressor(random_state = 0, max_depth=8, min_samples_leaf=10)
#     reg_y = DecisionTreeRegressor(random_state = 0)
    # fit model using data X, y
    reg_y.fit(X, y)
    # check number of features
#     print(reg_y.n_features_)
    # check feature importance 
    print(reg_y.feature_importances_)

#     print(reg_y.decision_path)
    # y_hat is the predicted value of y
    y_hat = reg_y.predict(X) 

    # save predicted y_hat as a pd dataframe and move its index to match the place in original df
    y_hat_df = pd.DataFrame(y_hat)
    y_hat_df.index += maxlag
    # save the predicted value into dataframe
    data['predicted_{}'.format(y_name)] = y_hat_df  

    # compuate mse
    reg_mse = mean_squared_error(y,y_hat)
    # compute residual value of y, y-y_hat, the residual value is the y in next round of loop
    if y_name == x_name:
        # learning rate is not in model 0
        y_residual = y - y_hat
        # apply leraning rate
    else:
        y_residual = y - (y_hat * v)
        
    data["{}res{}".format(y_name,x_name)] = y_residual

    # print mse, r^2, variance
    print("the mse is")
    print(reg_mse)
    print("regression score is")
#     print(r2_score(data['{}'.format(y_name)].iloc[3:], data['predicted_{}'.format(y_name)].iloc[3:]))
    # score is the r2_score, same results
    print(reg_y.score(X,y))
    r2 = reg_y.score(X,y)
#     print("var_reg is")
  #   print(df['predicted_{}'.format(y_name)].var(ddof=0))
#     var_reg = df['predicted_{}'.format(y_name)].var(ddof=0)  

    #print explained_variance_score
    print("explained_variance_score")
    variance_score = explained_variance_score(y,y_hat)
    print(variance_score)

#     print(data.head(10))
    return reg_mse,reg_y.score(X,y),variance_score,r2


def boosting(x_list, y_name, maxlag):
  
    # loop through each variable in the list
    temp_y_name = y_name
    mse_arr = []
    r2_arr = []
    
    predicted_name_list = []
    
    for pivot_x in range(0,len(x_list)):
        print("=========this is regression round {}=========".format(pivot_x+1))

        # save return value of regression in res_list
        res_list = regression(df, x_list[pivot_x],y_name,3)  
        
        # save predicted column name as a list
        predicted_name_list.append('predicted_{}'.format(y_name))
        
        # build y_name such as x1resx1, which means x1 substacts x1_hat, res means residual
        y_name = str(y_name) +"res"+ str(x_list[pivot_x])
        
        # example: [0.7614110755692759, 0.6019695603895466, 0.4941602516989991, 0.36284165024184334]
        mse_arr.append(res_list[0])
        r2_arr.append(res_list[3])
    
    return mse_arr,predicted_name_list,r2_arr,maxlag


def causality_test(boosting_result_list):
    
    mse_arr = boosting_result_list[0]
    name_list = boosting_result_list[1]
    r2_arr =  boosting_result_list[2]
    maxlag = boosting_result_list[3]
    
    print('------------Causalilty Test Criterias------------')
    
    # mse_y means the mse to predict y using all other varaibles except for the causing variable

    mse_y = mse_arr[len(mse_arr)-2]
#     print(mse_arr[len(mse_arr)-1])
    mse_all = mse_arr[len(mse_arr)-1]

    print("mse before adding causing variable is ")
    print(mse_y)
    print("mse of all variables is")
    print(mse_all)
    print("\n!!!!!!!!!!!!!!!!!!!!!!!")
    print("change of mse (ratio)")
#     mse_change = mse_y/mse_all
    mse_change = ((mse_y-mse_all)/(3-2))/(mse_all/(999-3))
    
    print(np.log(mse_change))
    print("!!!!!!!!!!!!!!!!!!!!!!!\n")
    
    print("~~~~~~~~~~~~~~~~~")
    print("the F-score is")
    f_score = ((mse_y-mse_all)/mse_all)*((n-k*maxlag)/maxlag)
    print(n-k*maxlag)
    print(maxlag)
    print(k*maxlag)
    print(f_score)
    p_value = scipy.stats.f.sf(f_score, maxlag, n-k*maxlag)
    print("the p_value is")
    print(p_value)
    print("~~~~~~~~~~~~~~~~~")
    
    
    df['pred_y'] = df[name_list[0]]
    for key in range(1, len(name_list)):
        df['pred_y'] += df[name_list[key]]

    df['last_step'] = df['pred_y'] - df[name_list[len(name_list)-1]]

    # df['step_3'] = df['predicted_x3'] + df['predicted_x3resx3'] + df['predicted_x3resx3resx4']

    r2_y = r2_arr[len(r2_arr)-2]
#     print(mse_arr[len(mse_arr)-1])
    r2_all = r2_arr[len(r2_arr)-1]
  
    print("r_square_last is")
    print(r2_y)
    print("r_square_final is ")
    print(r2_all)
    print("\n!!!!!!!!!!!!!!!!!!!!!!!")
    print("r-square change")
    r_square_change = abs(r2_all-r2_y)/r2_y
    print(r_square_change)
    print("!!!!!!!!!!!!!!!!!!!!!!!\n")
    
    
#     # draw graph if var_change >0.05  -- to do
#     if var_change > 0.05:
#         g.edge(,,label = " {} ".format(temp_lag))

    return name_list



## Tests of causality should exist


###: x1→x3|x4, x2

mse stable   r2 not stable if change x4 x2 position

0.36683870563049154

In [16]:
causality_test(boosting(["x3","x4","x2","x1"], "x3", 3))
# print(g)
# print(g.view())
# g

=========this is regression round 1=========
y is 
x3
X are
x3
[0.35730607 0.39614941 0.24654452]
the mse is
0.9535257455964228
regression score is
0.1478716027056236
explained_variance_score
0.1478716027056235
=========this is regression round 2=========
y is 
x3resx3
X are
x4
[0.24142623 0.23311102 0.52546275]
the mse is
0.8244165683546292
regression score is
0.13540187859430775
explained_variance_score
0.13540187859430775
=========this is regression round 3=========
y is 
x3resx3resx4
X are
x2
[0.32268388 0.3029345  0.37438162]
the mse is
0.79282985695766
regression score is
0.16628165484578838
explained_variance_score
0.16628165484578827
=========this is regression round 4=========
y is 
x3resx3resx4resx2
X are
x1
[0.57143821 0.19699852 0.23156327]
the mse is
0.7537504415453963
regression score is
0.2047450040342358
explained_variance_score
0.2047450040342358
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.79282985695766
mse of all variabl

['predicted_x3',
 'predicted_x3resx3',
 'predicted_x3resx3resx4',
 'predicted_x3resx3resx4resx2']

### X2→x3
 change x1 x4 cause large difference between r2, mse similar
 
 0.3127685509289654


In [17]:
causality_test(boosting(["x3","x4","x1","x2"], "x3", 3))

=========this is regression round 1=========
y is 
x3
X are
x3
[0.35730607 0.39614941 0.24654452]
the mse is
0.9535257455964228
regression score is
0.1478716027056236
explained_variance_score
0.1478716027056235
=========this is regression round 2=========
y is 
x3resx3
X are
x4
[0.24142623 0.23311102 0.52546275]
the mse is
0.8244165683546292
regression score is
0.13540187859430775
explained_variance_score
0.13540187859430775
=========this is regression round 3=========
y is 
x3resx3resx4
X are
x1
[0.5716237  0.19668936 0.23168693]
the mse is
0.756220975822083
regression score is
0.20477855998937255
explained_variance_score
0.20477855998937244
=========this is regression round 4=========
y is 
x3resx3resx4resx1
X are
x2
[0.32223146 0.30318333 0.37458521]
the mse is
0.7897028939060715
regression score is
0.1661719571585739
explained_variance_score
0.1661719571585739
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.756220975822083
mse of all varia

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in log


['predicted_x3',
 'predicted_x3resx3',
 'predicted_x3resx3resx4',
 'predicted_x3resx3resx4resx1']

### X4→x3

mse stable , r2 not   

0.28674225600921976

In [18]:
causality_test(boosting(["x3","x1","x2","x4"], "x3", 3))

=========this is regression round 1=========
y is 
x3
X are
x3
[0.35730607 0.39614941 0.24654452]
the mse is
0.9535257455964228
regression score is
0.1478716027056236
explained_variance_score
0.1478716027056235
=========this is regression round 2=========
y is 
x3resx3
X are
x1
[0.57165662 0.19688715 0.23145624]
the mse is
0.7582104915841105
regression score is
0.20483479855087094
explained_variance_score
0.20483479855087094
=========this is regression round 3=========
y is 
x3resx3resx1
X are
x2
[0.32173448 0.30344191 0.37482361]
the mse is
0.7918530721269036
regression score is
0.16615356420709948
explained_variance_score
0.16615356420709948
=========this is regression round 4=========
y is 
x3resx3resx1resx2
X are
x4
[0.24181048 0.23341862 0.5247709 ]
the mse is
0.8183042790314508
regression score is
0.13544097688633827
explained_variance_score
0.13544097688633827
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.7918530721269036
mse of all v

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in log


['predicted_x3',
 'predicted_x3resx3',
 'predicted_x3resx3resx1',
 'predicted_x3resx3resx1resx2']

## Tests of no causality


### X4→x1

In [20]:
causality_test(boosting(["x1","x2","x3","x4"], "x1", 3))

=========this is regression round 1=========
y is 
x1
X are
x1
[0.37871176 0.39067219 0.23061605]
the mse is
0.7385118636176096
regression score is
0.23685132649374963
explained_variance_score
0.23685132649374963
=========this is regression round 2=========
y is 
x1resx1
X are
x2
[0.38084139 0.45509787 0.16406075]
the mse is
0.6880026478730433
regression score is
0.06839323541418318
explained_variance_score
0.06839323541418307
=========this is regression round 3=========
y is 
x1resx1resx2
X are
x3
[0.34622284 0.25774841 0.39602875]
the mse is
0.6612108425982597
regression score is
0.10345110695170168
explained_variance_score
0.10345110695170168
=========this is regression round 4=========
y is 
x1resx1resx2resx3
X are
x4
[0.41297328 0.20708872 0.379938  ]
the mse is
0.6549111545455607
regression score is
0.11016108797576131
explained_variance_score
0.11016108797576118
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.6612108425982597
mse of all

['predicted_x1',
 'predicted_x1resx1',
 'predicted_x1resx1resx2',
 'predicted_x1resx1resx2resx3']

### X2→x1

In [21]:
causality_test(boosting(["x1","x4","x3","x2"], "x1", 3))

=========this is regression round 1=========
y is 
x1
X are
x1
[0.37871176 0.39067219 0.23061605]
the mse is
0.7385118636176096
regression score is
0.23685132649374963
explained_variance_score
0.23685132649374963
=========this is regression round 2=========
y is 
x1resx1
X are
x4
[0.41291988 0.20923078 0.37784934]
the mse is
0.6571895622624121
regression score is
0.11011644546485577
explained_variance_score
0.11011644546485577
=========this is regression round 3=========
y is 
x1resx1resx4
X are
x3
[0.34650752 0.25755213 0.39594034]
the mse is
0.6606543467208774
regression score is
0.10346026657218044
explained_variance_score
0.10346026657218055
=========this is regression round 4=========
y is 
x1resx1resx4resx3
X are
x2
[0.38103148 0.4551683  0.16380022]
the mse is
0.6850570802878432
regression score is
0.06842660452163918
explained_variance_score
0.06842660452163929
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.6606543467208774
mse of all

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: RuntimeWarning: invalid value encountered in log


['predicted_x1',
 'predicted_x1resx1',
 'predicted_x1resx1resx4',
 'predicted_x1resx1resx4resx3']

### X3→x1

In [22]:
causality_test(boosting(["x1","x4","x2","x3"], "x1", 3))

=========this is regression round 1=========
y is 
x1
X are
x1
[0.37871176 0.39067219 0.23061605]
the mse is
0.7385118636176096
regression score is
0.23685132649374963
explained_variance_score
0.23685132649374963
=========this is regression round 2=========
y is 
x1resx1
X are
x4
[0.41291988 0.20923078 0.37784934]
the mse is
0.6571895622624121
regression score is
0.11011644546485577
explained_variance_score
0.11011644546485577
=========this is regression round 3=========
y is 
x1resx1resx4
X are
x2
[0.38077321 0.45531762 0.16390917]
the mse is
0.6865161879834034
regression score is
0.0683645037325944
explained_variance_score
0.0683645037325944
=========this is regression round 4=========
y is 
x1resx1resx4resx2
X are
x3
[0.34643541 0.2573016  0.39626299]
the mse is
0.6597099761693149
regression score is
0.10352220638211583
explained_variance_score
0.10352220638211584
------------Causalilty Test Criterias------------
mse before adding causing variable is 
0.6865161879834034
mse of all v

['predicted_x1',
 'predicted_x1resx1',
 'predicted_x1resx1resx4',
 'predicted_x1resx1resx4resx2']